In [1]:
from pyspark.sql import SparkSession, functions as F

In [2]:
import findspark
import os
import time

In [3]:
from pyspark.sql.types import *

In [4]:
findspark.init("/opt/manual/spark")

In [5]:
spark = (SparkSession.builder
                    .config("spark.sql.shuffle.partitions", 5)
                    .config("spark.driver.memory", "16g")
                    .master("local[4]")
                    .appName("Pre_Proccessing_refined")
                    .getOrCreate())

2022-11-23 02:28:41,618 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
rootdir = "/home/train/datasets/sensor_raw_dataset/KETI/"

file_path_list = []
directory_names = []
names=[]
dirpaths=[]

for (dirpath, dirnames, filenames) in os.walk(rootdir):#topdown incelenecek...
    numbers=range(len(filenames))
    for f in numbers:
        if "csv" in filenames[f]:
            file_path_list.append(os.path.join(dirpath, filenames[f]))
            dirpaths.append(dirpath)
names=filenames

for dosya in os.listdir(rootdir):#if "csv" in filenames[f]
    directory_names.append(dosya)

directory_names = [directory for directory in directory_names if "txt" not in directory]

In [7]:
schema_co2 = StructType([StructField('ts_min_bignt',StringType(),True),
                         StructField('co2',FloatType(),True)])

schema_humidity = StructType([StructField('tmstmp1',StringType(),True),
                              StructField('humidity',FloatType(),True)])

schema_light = StructType([StructField('tmstmp2',StringType(),True),
                           StructField('light',FloatType(),True)])

schema_pir = StructType([StructField('tmstmp3',StringType(),True),
                         StructField('pir',FloatType(),True)])
                                       
schema_temperature = StructType([StructField('tmstmp4',StringType(),True),
                                 StructField('temperature',FloatType(),True)])

In [8]:
list_schema = [schema_co2, schema_humidity,schema_light,schema_pir, schema_temperature]

In [9]:
start_time = time.time()


for n in range(0,len(dirpaths),5):
    for i in range(len(names)):
        df=(spark.read
             .option("header", False)
             .format("csv")
             .schema(list_schema[i])
             .load(f"file://{dirpaths[n]}/{names[i]}")
           )
        if i == 0:
            df_co2=df.withColumn("room",F.lit((f"file://{dirpaths[n]}").split("/")[-1]))\
                    .withColumn("event_ts_min",F.to_timestamp(F.from_unixtime(F.col("ts_min_bignt"),'yyyy-MM-dd HH:mm:ss')))
        elif i == 1:
            df_humidity =df
        elif i == 2:
            df_light = df
        elif i == 3:
            df_pir = df
        else:
            df_temperature  = df

    df_ch=df_co2.join(df_humidity,(df_co2["ts_min_bignt"] == df_humidity["tmstmp1"]),"inner")
    df_chl = df_ch.join(df_light, (df_ch["ts_min_bignt"] == df_light["tmstmp2"]),"inner")
    df_chlp = df_chl.join(df_pir, (df_chl["ts_min_bignt"] == df_pir["tmstmp3"]),"inner")
    df_full= df_chlp.join(df_temperature, (df_chlp["ts_min_bignt"] == df_temperature["tmstmp4"]),"inner")

    df_full.repartition(1).write\
                      .format("csv")\
                      .mode("append")\
                      .option("header", True)\
                      .save("file:///home/train/datasets/sensor_raw_dataset/output/")

print("----- %s secs -----" %(time.time() - start_time))

----- 40.60904335975647 secs -----


In [10]:
df_read_from_csv= spark.read.format("csv").option("header", "true").load("file:///home/train/datasets/sensor_raw_dataset/output/*")

In [11]:
df_read_from_csv=df_read_from_csv.select("event_ts_min","ts_min_bignt","room","co2","light","temperature","humidity","pir")

In [12]:
print((df_read_from_csv.count(), len(df_read_from_csv.columns)))

(135386, 8)


In [13]:
df_read_from_csv.repartition(1).write\
                      .format("csv")\
                      .mode("append")\
                      .option("header", True)\
                      .save("file:///home/train/datasets/sensor_raw_dataset/datagen_input/")

In [14]:
df_read_from_csv.show()

+--------------------+------------+----+-----+-----+-----------+--------+---+
|        event_ts_min|ts_min_bignt|room|  co2|light|temperature|humidity|pir|
+--------------------+------------+----+-----+-----+-----------+--------+---+
|2013-08-24T02:05:...|  1377299105| 666|437.0|148.0|      24.14|   49.39|0.0|
|2013-08-24T02:05:...|  1377299126| 666|442.0|149.0|      24.14|   49.42|0.0|
|2013-08-24T02:05:...|  1377299130| 666|442.0|146.0|      24.14|   49.42|0.0|
|2013-08-24T02:05:...|  1377299138| 666|442.0|149.0|      24.14|   49.42|0.0|
|2013-08-24T02:05:...|  1377299148| 666|448.0|148.0|      24.14|   49.42|0.0|
|2013-08-24T02:05:...|  1377299158| 666|449.0|150.0|      24.13|   49.42|0.0|
|2013-08-24T02:06:...|  1377299168| 666|445.0|150.0|      24.13|   49.42|0.0|
|2013-08-24T02:06:...|  1377299178| 666|445.0|150.0|      24.13|   49.42|0.0|
|2013-08-24T02:06:...|  1377299188| 666|437.0|150.0|      24.13|   49.42|0.0|
|2013-08-24T02:06:...|  1377299198| 666|437.0|150.0|      24.12|